## Proposta

Este trabalho propôs a criação de um sistema utilizando algoritmos de aprendizagem de máquina para maximizar a métrica de hit ratio. [1](https://www.cloudflare.com/pt-br/learning/cdn/what-is-a-cache-hit-ratio/#:~:text=A%20cache%20hit%20ratio%20is,at%20fulfilling%20requests%20for%20content.)

$$\frac{\textrm{number of cache hits}}{\textrm{number of cache hits + number of cache misses}} = {\textrm{hit ratio}}{\textrm{    }}(1)$$ 

<br><br>
Os passos do sistema estão descritos no diagrama abaixo:
<center><img src="images/diagrama.png"></center>

_A. Obtenção dos dados_

Para avaliar o desempenho do sistema proposto foi utilizado um conjunto de dados sintéticos gerados utilizando o código utilizado no projeto DeepCache[2](https://github.com/eman-ramadan/deepcache_netai2018). Os dados gerados seguem a distribuição Zipf para simular a popularidade de cada item ao longo do tempo. O conjunto de dados gerados possue duas colunas, Object_ID e request_time. Para o trabalho foi adicionando uma nova coluna ao conjunto de dados chamado size.

* **Object_ID**: Número inteiro representando o ID do objeto que está sendo requisitado.
* **request_time**: Tempo incremental em segundos representando o momento que o Object_ID foi requisitado.
* **size**: Tamanho do Object_ID em bytes seguindo a distribuição normal.

Na etapa de pré-processamento são adicionadas novas colunas para serem utilizadas como _features_ nos algoritmos de aprendizagem de máquina. As novas colunas são:
* **time**: Conversão da coluna request_time para o formato data e hora, começando em 01/08/2018.
* **hour**: Agrupamento da coluna request_time em intervalos de 3600 segundos e atribuido a cada intervalo um valor representando a hora que ocorreu a requisição.
* **total_requests_1H**: Janela deslizante de hora em hora incremental com o número de acessos do object_ID.
* **mean_requests_2H**: Janela deslizante de duas horas com o valor médio da coluna total_requests_1H.
* **count**: Total de acessos que um object_ID recebeu.

Por fim na etapa de pré-processamento é criada uma coluna chamada **cache** para classificar a requisição. Caso o valor dela seja 1 significa que o _object_ID_ tem uma maior probilidade de ser um item popular e que já deva estar ou precise estar em _cache_ e caso o valor seja 0 significa que o item tem uma probilidade menor de ser popular e que não precisa ser armazenado na cache.
O método adotado para a classificação foi se o _object_ID_ da linha atual aparece pelo menos uma vez nos próximos 15 _frames_ do conjunto de dados.


_C. Algoritmos de classificação utilizados_

Para este trabalho foi utilizada integralmente a linguagem de programação _Python_.
_Python_ é uma linguagem de programação de alto nível, multiparadigma e interpretada. Devido ao fato de ser uma linguagem fácil de ser apredendida e por todo seu ecossistema, tanto de bibliotecas como da comunidade, ela se tornou uma das linguagens mais utilizadas para tarefas de análise de dados, aprendizagem de máquina e inteligência artificial. [2](https://marutitech.com/python-data-science/) [3](https://www.davekuhlman.org/python_book_01.pdf)

Os algoritimos de classificação escolhidos para esse trabalho foram: _Random Forest_, _Gradient Boosting_ e _Decision Tree_. Foram utilizados os algoritmos implementados no pacote _Scikit-learn_ neste trabalho. O _Scikit-learn_ é uma biblioteca da linguagem de programação _Python_ que possui vários algoritmos de aprendizagem de máquina para problemas supervisionados e não supervisionados. [4](https://jmlr.org/papers/v12/pedregosa11a.html)


_D. Algoritmos de cache utilizados_

Para este trabalho foram selecionados dois algoritmos de _cache_: 

* **_Least Recently Used_ (LRU)**: Remove o último objeto que está a mais tempo sem ser utilizado quando o cache excede o seu tamanho máximo.
* **_Least Frequently Used_ (LFU)**: Remoeve o objeto que é menos utilizado baseado em sua frequência. Ele utiliza um contador para cada objeto para contar a frequência de cada objeto, quando o cache excede o seu tamanho máximo ele remove a página menos frequente.

## Experimentos

Os experimentos foram dividios em três partes. Primeiramente o treinamento e avaliação dos algoritmos de classificação no conjunto de dados de treinamento com o objetivo de avaliar o algoritmo de classificação com melhor desempenho. Posteriormente, no conjunto de dados de testes, foi avaliada a métrica de _hit ratio_ dos algortimos de _cache_ e dos algoritmos de aprendizagem de máquina em conjunto com os algoritmos de _cache_ com objetivo de saber se existe um ganho de _hit ratio_ na utilização de um algoritmo de classificação antes de rodar o algoritmo de cache e em qual dos algoritmos, LRU ou LFU, esse ganho é maior. Por fim foi avaliado o tempo médio gasto na execução dos algoritmos de _cache_ e dos algoritmos de classificação em conjunto do algoritmo de _cache_ para avaliar a aplicabilidade da técnica utilizando o algoritmo de classificação no mundo real.


_A. Métricas_

Para avaliar os modelos gerados pelos algoritmos de classificação foram utilizadas métricas calculadas a partir de uma matriz de confusão [5](https://medium.com/@vitorborbarodrigues/m%C3%A9tricas-de-avalia%C3%A7%C3%A3o-acur%C3%A1cia-precis%C3%A3o-recall-quais-as-diferen%C3%A7as-c8f05e0a513c).
A matriz de confusão indica em formato de tabela os erros e acertos do modelo. A tabela abaixo demostra um exemplo de uma matriz de confusão: 

|   | Valor previsto positivo | Valor previsto negativo |
|---|---|---|
| **Valor real positivo** |  <center>Verdadeiro Positivo<br>(VP)</center> |  <center>Falso Negativo<br>(FN)</center> |
| **Valor real negativo** |  <center>Falso Positivo<br>(FP)</center> | <center>Verdadeiro Negativo<br>(VN)</center> |

<center> Matriz de confusão</center>


* **Verdadeiro Positivo (VP)**: Classificação correta dos valores positivos previstos.
* **Falso Negativo (FN)**: Classificação errada dos valores positivos previstos.
* **Falso Positivo (FP)**: Classificação errada dos valores negativos previstos.
* **Verdadeiro Negativo (VN)**: Classificação correta dos valores negativos previstos.


A partir desses valores é calculada as seguintes métricas para avaliação dos modelos:

* **Acurácia**: Indica a perfomance geral que o modelo classificou corretamente, sua fórmula é:
$$\frac{{VP} + {VN}}{{VP} + {VN} + {FP} + {FN}}$$ 


* **Precisão**: Indica a quantidade das classificações positivas que o modelo fez e quantas estão corretas, sua formulá é:
$$\frac{{VP}}{{VP} + {FP}}$$ 


* **_Recall_**: Indica a quantidade das classificações positivas corretas, sua formulá é:
$$\frac{{VP}}{{VP} + {FN}}$$ 


* **_F1-Score_**: Média harmônica entre precisão e _recall_, sua formulá é:
$$2 * \frac{{Precisão} * {Recall}}{{Precisão} + {Recall}}$$



Posteriormente para avaliar o desemepenho dos modelos em conjunto com os algoritmos de cache no sistema proposto foi utilizado a métrica de _hit ratio_ da equação (1).
No fim para avaliar o tempo médio do sistema, foram selecionados 2000 mil requisições do conjunto de teste e medido o tempo de cada requisição no sistema apenas com os algoritmos de cache e posteriormente nos sistemas propostos com os algoritmos de classificação.

## Parte prática

In [ ]:
import numpy as np
import pandas as pd
import math, random
from scipy import special
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime, timedelta

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
np.random.seed(0)

In [ ]:
df = pd.read_csv("final_datasets/df_ml.csv")
df = df.set_index("time")
df['cache'] = df['cache'].astype(int)

display(df.head(18))

In [ ]:
len(df)

Separação dos dados em dados de treinamento e dados de testes

In [ ]:
train, test = train_test_split(df, test_size=0.3, random_state=42, shuffle=False)

print('Número de observações nos dados de treinamento:', len(train))
print('Número de observações nos dados de testes:',len(test))

Escolhendo as _features_ que serão utilizadas pelos modelos para prever a classificação da coluna _cache_

In [ ]:
features = df.columns[1:6]
print('Features escolhidas:', ', '.join(features))

In [ ]:
y, uniques = pd.factorize(train['cache'])

### Execução e avaliação dos algoritmos classificadores

#### Decision Tree

In [ ]:
clf_tree = DecisionTreeClassifier(random_state=0, max_depth=5)
clf_tree.fit(train[features], y)

In [ ]:
# Realiza a previsão da coluna cache no conjunto de dados de testes
preds_tree = uniques[clf_tree.predict(test[features])]

In [ ]:
pd.crosstab(test['cache'], preds_tree, rownames=['Valores reais'], colnames=['Valores previstos'])

In [ ]:
def calcula_metricas_classificadores(test, preds_algo):
  accuracy = round(accuracy_score(test['cache'], preds_algo), 2) * 100
  precision = round(precision_score(test['cache'], preds_algo), 2) * 100
  recall = round(recall_score(test['cache'], preds_algo), 4) * 100
  f1 = round(f1_score(test['cache'], preds_algo), 2) * 100

  print(f"Acurácia: {accuracy}%")
  print(f"Precisão: {precision}%")
  print(f"Recall: {recall}%")
  print(f"F1-Score: {f1}%")

In [ ]:
calcula_metricas_classificadores(test, preds_tree)

In [ ]:
importances = pd.DataFrame({'feature':features,'Importância':np.round(clf_tree.feature_importances_,3)})
importances = importances.sort_values('Importância',ascending=False).set_index('feature')
display(importances)

#### Random Forest

In [ ]:
clf_rf = RandomForestClassifier(n_jobs=12, random_state=0)
clf_rf.fit(train[features], y)

In [ ]:
# Realiza a previsão da coluna cache no conjunto de dados de testes
preds_rf = uniques[clf_rf.predict(test[features])]

In [ ]:
pd.crosstab(test['cache'], preds_rf, rownames=['Valores reais'], colnames=['Valores previstos'])

In [ ]:
calcula_metricas_classificadores(test, preds_rf)

In [ ]:
importances = pd.DataFrame({'feature':features,'Importância':np.round(clf_rf.feature_importances_,3)})
importances = importances.sort_values('Importância',ascending=False).set_index('feature')
display(importances)

#### Gradient Boosting

In [ ]:
clf_gbc = GradientBoostingClassifier(random_state=0)
clf_gbc.fit(train[features], y)

In [ ]:
# Realiza a previsão da coluna cache no conjunto de dados de testes
preds_gbc = uniques[clf_gbc.predict(test[features])]

In [ ]:
pd.crosstab(test['cache'], preds_gbc, rownames=['Valores reais'], colnames=['Valores previstos'])

In [ ]:
calcula_metricas_classificadores(test, preds_gbc)

In [ ]:
importances = pd.DataFrame({'feature':features,'Importância':np.round(clf_gbc.feature_importances_,3)})
importances = importances.sort_values('Importância',ascending=False).set_index('feature')
display(importances)

Dentre os algoritmos propostos o algoritmo de classificação o que se saiu melhor nos dados de testes foi o algoritmo de _Decision Tree_ em segundo lugar o _Gradient Boosting_ e por fim o _Random Forest_. Embora o _Random Forest_ foi o último colocado ele foi o que apresentou o maior _recall_ dentre os três algoritmos propostos.

### Execução e avaliação dos algoritmos de cache com e sem os classificadores

In [ ]:
from lru import LRU
from lfu import LFU

#### Execução apenas dos algoritmos de cache

In [ ]:
def executa_apenas_cache(algoritmo_cache, capacidade_cache, df):
  cache = algoritmo_cache(capacity=capacidade_cache)
  cache_status = {}
  cache_counter = []
  c = 0 

  events = df['object_ID'].values

  for event_key in events:
    if event_key not in cache_status:
      cache_status[event_key] = {
          'found_in_cache': 0,
          'n_requested': 0
      }

    item = cache.get(event_key)
    if item != -1:
      c += 1
      cache_status[event_key]['found_in_cache'] += 1
    else:
      cache.put(event_key, 1)

    cache_counter.append(c)
    cache_status[event_key]['n_requested'] += 1
    cache_status[event_key]['hit_ratio'] = round(cache_status[event_key]['found_in_cache'] / cache_status[event_key]['n_requested'], 2)

  final_hit_ratios = pd.DataFrame(cache_status).transpose().sort_values("hit_ratio", ascending=False)
  
  
  return cache_counter, final_hit_ratios

In [ ]:
counter_LRU, df_hit_ratios_LRU = executa_apenas_cache(LRU, 15, test)
found_in_cache_total_LRU = df_hit_ratios_LRU['found_in_cache'].sum()
n_requested_total_LRU = df_hit_ratios_LRU['n_requested'].sum()
hit_ratio_LRU = round(( found_in_cache_total_LRU / n_requested_total_LRU ) * 100, 2)

print('Hit Ratio LRU:', hit_ratio_LRU)

In [ ]:
counter_LFU, df_hit_ratios_LFU = executa_apenas_cache(LFU, 15, test)
found_in_cache_total_LFU = df_hit_ratios_LFU['found_in_cache'].sum()
n_requested_total_LFU = df_hit_ratios_LFU['n_requested'].sum()
hit_ratio_LFU = round(( found_in_cache_total_LFU / n_requested_total_LFU ) * 100, 2)

print('Hit Ratio LFU:', hit_ratio_LFU)

In [ ]:
figure(num=None, figsize=(12, 6), dpi=80, facecolor='w', edgecolor='k')

plt.plot(counter_LRU, label="LRU")
plt.plot(counter_LFU, label="LFU")
plt.xlabel('Número total de requisições')
plt.ylabel('Total de requisições encontradas em cache')
plt.legend()
plt.show()

#### Execução do sistema proposto(algoritmo de classificação + algoritmo de cache)

In [ ]:
def executa_sistema_proposto(algoritmo_cache, capacidade_cache, df):
  cache = algoritmo_cache(capacity=capacidade_cache)
  cache_status = {}
  cache_counter = []
  c = 0 


  for i, row in df.iterrows():
    event_key = row['object_ID']
    can_cache = row['preds']
    
    if not can_cache:
      continue
      
    if event_key not in cache_status:
      cache_status[event_key] = {
          'found_in_cache': 0,
          'n_requested': 0
      }
    
    item = cache.get(event_key)
    if item != -1:
      c += 1
      cache_status[event_key]['found_in_cache'] += 1
    else:
      cache.put(event_key, 1)

    cache_counter.append(c)
    cache_status[event_key]['n_requested'] += 1
    cache_status[event_key]['hit_ratio'] = round(cache_status[event_key]['found_in_cache'] / cache_status[event_key]['n_requested'], 2)

  final_hit_ratios = pd.DataFrame(cache_status).transpose().sort_values("hit_ratio", ascending=False)
  
  
  return cache_counter, final_hit_ratios

#### Sistema proposto utilizando Decision Trees

In [ ]:
test_tree = test.copy()
test_tree['preds'] = preds_tree


counter_LRU_DT, df_hit_ratios_LRU_DT = executa_sistema_proposto(LRU, 15,test_tree)
found_in_cache_total_LRU_DT = df_hit_ratios_LRU_DT['found_in_cache'].sum()
n_requested_total_LRU_DT = df_hit_ratios_LRU_DT['n_requested'].sum()
hit_ratio_LRU_DT = round(( found_in_cache_total_LRU_DT / n_requested_total_LRU_DT ) * 100, 2)

print('Hit Ratio LRU + Decision Tree:', hit_ratio_LRU_DT)

In [ ]:
test_tree = test.copy()
test_tree['preds'] = preds_tree


counter_LFU_DT, df_hit_ratios_LFU_DT = executa_sistema_proposto(LFU, 15, test_tree)
found_in_cache_total_LFU_DT = df_hit_ratios_LFU_DT['found_in_cache'].sum()
n_requested_total_LFU_DT = df_hit_ratios_LFU_DT['n_requested'].sum()
hit_ratio_LFU_DT = round(( found_in_cache_total_LFU_DT / n_requested_total_LFU_DT ) * 100, 2)

print('Hit Ratio LFU + Decision Tree:', hit_ratio_LFU_DT)

#### Sistema proposto utilizando Random Forest

In [ ]:
test_rf = test.copy()
test_rf['preds'] = preds_rf


counter_LRU_RF, df_hit_ratios_LRU_RF = executa_sistema_proposto(LRU, 15, test_rf)
found_in_cache_total_LRU_RF = df_hit_ratios_LRU_RF['found_in_cache'].sum()
n_requested_total_LRU_RF = df_hit_ratios_LRU_RF['n_requested'].sum()
hit_ratio_LRU_RF = round(( found_in_cache_total_LRU_RF / n_requested_total_LRU_RF ) * 100, 2)

print('Hit Ratio LRU + Random Forest:', hit_ratio_LRU_RF)

In [ ]:
test_rf = test.copy()
test_rf['preds'] = preds_rf


counter_LFU_RF, df_hit_ratios_LFU_RF = executa_sistema_proposto(LFU, 15, test_rf)
found_in_cache_total_LFU_RF = df_hit_ratios_LFU_RF['found_in_cache'].sum()
n_requested_total_LFU_RF = df_hit_ratios_LFU_RF['n_requested'].sum()
hit_ratio_LFU_RF = round(( found_in_cache_total_LFU_RF / n_requested_total_LFU_RF ) * 100, 2)

print('Hit Ratio LFU + Random Forest:', hit_ratio_LFU_RF)

#### Sistema proposto utilizando Gradient Boosting

In [ ]:
test_gbc = test.copy()
test_gbc['preds'] = preds_gbc


counter_LRU_GBC, df_hit_ratios_LRU_GBC = executa_sistema_proposto(LRU, 15, test_gbc)
found_in_cache_total_LRU_GBC = df_hit_ratios_LRU_GBC['found_in_cache'].sum()
n_requested_total_LRU_GBC = df_hit_ratios_LRU_GBC['n_requested'].sum()
hit_ratio_LRU_GBC = round(( found_in_cache_total_LRU_GBC / n_requested_total_LRU_GBC ) * 100, 2)

print('Hit Ratio LRU + Gradient Boosting:', hit_ratio_LRU_GBC)

In [ ]:
test_gbc = test.copy()
test_gbc['preds'] = preds_gbc


counter_LFU_GBC, df_hit_ratios_LFU_GBC = executa_sistema_proposto(LFU, 15, test_gbc)
found_in_cache_total_LFU_GBC = df_hit_ratios_LFU_GBC['found_in_cache'].sum()
n_requested_total_LFU_GBC = df_hit_ratios_LFU_GBC['n_requested'].sum()
hit_ratio_LFU_GBC = round(( found_in_cache_total_LFU_GBC / n_requested_total_LFU_GBC ) * 100, 2)

print('Hit Ratio LFU + Gradient Boosting:', hit_ratio_LFU_GBC)

In [ ]:
print(n_requested_total_LRU_DT)
print(n_requested_total_LFU_DT)
print(n_requested_total_LRU_RF)
print(n_requested_total_LFU_RF)
print(n_requested_total_LRU_GBC)
print(n_requested_total_LFU_GBC)

In [ ]:
figure(num=None, figsize=(15, 12), dpi=80, facecolor='w', edgecolor='k')

plt.plot(counter_LRU, label="LRU")
plt.plot(counter_LFU, label="LFU")

plt.plot(counter_LRU_DT, label="LRU + DT")
plt.plot(counter_LFU_DT, label="LFU + DT")
plt.plot(counter_LRU_RF, label="LRU + RF")
plt.plot(counter_LFU_RF, label="LFU + RF")
plt.plot(counter_LRU_GBC, label="LRU + GB")
plt.plot(counter_LFU_GBC, label="LFU + GB")
# plt.title("Algoritmos utilizando LRU")
plt.title("Comparação dos algoritmos tradicionais e dos sistemas propostos")
plt.xlabel('Número total de requisições', fontsize=12)
plt.ylabel('Total de requisições encontradas em cache', fontsize=12)
plt.legend()
plt.show()

In [ ]:
figure(num=None, figsize=(15, 12), dpi=80, facecolor='w', edgecolor='k')

# plt.plot(counter_LRU, label="LRU")
plt.plot(counter_LFU, label="LFU")

# plt.plot(counter_LRU_DT, label="LRU + DT")
plt.plot(counter_LFU_DT, label="LFU + DT")
# plt.plot(counter_LRU_RF, label="LRU + RF")
plt.plot(counter_LFU_RF, label="LFU + RF")
# plt.plot(counter_LRU_GBC, label="LRU + GBC")
plt.plot(counter_LFU_GBC, label="LFU + GBC")
plt.title("Algoritmos utilizando LFU")
plt.xlabel('Número total de requisições')
plt.ylabel('Total de requisições encontradas em cache')
plt.legend()
plt.show()

----

In [ ]:
def executa_algoritmo_cache_benchmark(algoritmo_cache, capacidade_cache, df):
  rows = []
  for i, r in df.reset_index().iterrows():
    current_row = df.iloc[i:i+1].copy()

    rows.append(current_row)


  cache = algoritmo_cache(capacity=capacidade_cache)
  cache_status = {}

  cache_counter = []
  times = []
  c = 0

  total = len(rows)
  for i, current_row in enumerate(rows[:1000]):
    start_time = datetime.now()

    event_key = current_row['object_ID'].iloc[0]

    if event_key not in cache_status:
      cache_status[event_key] = {
          'found_in_cache': 0,
          'n_requested': 0
      }

    item = cache.get(event_key)
    if item != -1:
      c += 1
      cache_status[event_key]['found_in_cache'] += 1
    else:
      cache.put(event_key, 1)

    end_time = datetime.now()
    dt = end_time - start_time

    cache_counter.append(c)
    times.append(dt)
    
  return times

In [ ]:
times_LRU = executa_algoritmo_cache_benchmark(LRU, 15, test)
times_LFU = executa_algoritmo_cache_benchmark(LFU, 15, test)

In [ ]:
x_times_LRU = pd.DataFrame(times_LRU)
x_times_LRU[0] = x_times_LRU[0]  / np.timedelta64(1, 's')

figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
plt.title("Desempenho LRU por requisição")
plt.xlabel("Total de requisições")
plt.ylabel("Tempo em segundos")
plt.plot(x_times_LRU)

In [ ]:
df_times_LRU = x_times_LRU.describe().transpose()
df_times_LRU['algo'] = 'LRU'

x_times_LRU.describe()

In [ ]:
x_times_LFU = pd.DataFrame(times_LFU)
x_times_LFU[0] = x_times_LFU[0]  / np.timedelta64(1, 's')

figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
plt.title("Desempenho LFU por requisição")
plt.xlabel("Total de requisições")
plt.ylabel("Tempo em segundos")
plt.plot(x_times_LFU)

In [ ]:
df_times_LFU = x_times_LFU.describe().transpose()
df_times_LFU['algo'] = 'LFU'
x_times_LFU.describe()

In [ ]:
def executa_sistema_proposto_benchmark(algoritmo_cache, capacidade_cache, algo_cl, df):
  rows = []
  for i, r in df.reset_index().iterrows():
    current_row = df.iloc[i:i+1].copy()

    rows.append(current_row)


  cache = algoritmo_cache(capacity=capacidade_cache)
  cache_status = {}

  cache_counter = []
  times = []
  c = 0

  total = len(rows)
  for i, current_row in enumerate(rows[:1000]):
    start_time = datetime.now()

    pred = uniques[algo_cl.predict(current_row[features])]
    current_row['preds'] = pred

    event_key = current_row['object_ID'].iloc[0]
    can_cache = current_row['preds'].iloc[0]

    if not can_cache:
      continue

    if event_key not in cache_status:
      cache_status[event_key] = {
          'found_in_cache': 0,
          'n_requested': 0
      }

    item = cache.get(event_key)
    if item != -1:
      c += 1
      cache_status[event_key]['found_in_cache'] += 1
    else:
      cache.put(event_key, 1)

    end_time = datetime.now()
    dt = end_time - start_time

    cache_counter.append(c)
    times.append(dt)
    
  return times

In [ ]:
test_tree = test.copy()
test_tree['preds'] = preds_tree

times_LRU_DT = executa_sistema_proposto_benchmark(LRU, 15, clf_tree, test_tree)
times_LFU_DT = executa_sistema_proposto_benchmark(LFU, 15, clf_tree, test_tree)

In [ ]:
x_times_LRU_DT = pd.DataFrame(times_LRU_DT)
x_times_LRU_DT[0] = x_times_LRU_DT[0]  / np.timedelta64(1, 's')
figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
plt.title("Desempenho LRU + Decision Tree por requisição")

plt.xlabel("Total de requisições")
plt.ylabel("Tempo em segundos")
plt.plot(x_times_LRU_DT)

In [ ]:
df_times_LRU_DT = x_times_LRU_DT.describe().transpose()
df_times_LRU_DT['algo'] = 'LRU_DT'

x_times_LRU_DT.describe()

In [ ]:
x_times_LFU_DT = pd.DataFrame(times_LFU_DT)
x_times_LFU_DT[0] = x_times_LFU_DT[0]  / np.timedelta64(1, 's')

figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
plt.title("Desempenho LFU + Decision Tree por requisição")
plt.xlabel("Total de requisições")
plt.ylabel("Tempo em segundos")
plt.plot(x_times_LFU_DT)

In [ ]:
df_times_LFU_DT = x_times_LFU_DT.describe().transpose()
df_times_LFU_DT['algo'] = 'LFU_DT'

x_times_LFU_DT.describe()

In [ ]:
test_rf = test.copy()
test_rf['preds'] = preds_rf

times_LRU_RF = executa_sistema_proposto_benchmark(LRU, 15, clf_rf, test_tree)
times_LFU_RF = executa_sistema_proposto_benchmark(LFU, 15, clf_rf, test_tree)

In [ ]:
x_times_LRU_RF = pd.DataFrame(times_LRU_RF)
x_times_LRU_RF[0] = x_times_LRU_RF[0]  / np.timedelta64(1, 's')

figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
plt.title("Desempenho LRU + Random Forest por requisição")
plt.xlabel("Total de requisições")
plt.ylabel("Tempo em segundos")
plt.plot(x_times_LRU_RF)

In [ ]:
df_times_LRU_RF = x_times_LRU_RF.describe().transpose()
df_times_LRU_RF['algo'] = 'LRU_RF'

x_times_LRU_RF.describe()

In [ ]:
x_times_LFU_RF = pd.DataFrame(times_LFU_RF)
x_times_LFU_RF[0] = x_times_LFU_RF[0]  / np.timedelta64(1, 's')

figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
plt.title("Desempenho LFU + Random Forest por requisição")
plt.xlabel("Total de requisições")
plt.ylabel("Tempo em segundos")
plt.plot(x_times_LFU_RF)

In [ ]:
df_times_LFU_RF = x_times_LFU_RF.describe().transpose()
df_times_LFU_RF['algo'] = 'LFU_RF'

x_times_LFU_RF.describe()

In [ ]:
test_gbc = test.copy()
test_gbc['preds'] = preds_gbc

times_LRU_GBC = executa_sistema_proposto_benchmark(LRU, 15, clf_gbc, test_gbc)
times_LFU_GBC = executa_sistema_proposto_benchmark(LFU, 15, clf_gbc, test_gbc)

In [ ]:
x_times_LRU_GBC = pd.DataFrame(times_LRU_GBC)
x_times_LRU_GBC[0] = x_times_LRU_GBC[0]  / np.timedelta64(1, 's')

figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
plt.title("Desempenho LRU + Gradient Boosting por requisição")
plt.xlabel("Total de requisições")
plt.ylabel("Tempo em segundos")
plt.plot(x_times_LRU_GBC)

In [ ]:
df_times_LRU_GBC = x_times_LRU_GBC.describe().transpose()
df_times_LRU_GBC['algo'] = 'LRU_GBC'

x_times_LRU_GBC.describe()

In [ ]:
x_times_LFU_GBC = pd.DataFrame(times_LFU_GBC)
x_times_LFU_GBC[0] = x_times_LFU_GBC[0]  / np.timedelta64(1, 's')

figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
plt.title("Desempenho LFU + Gradient Boosting por requisição")
plt.xlabel("Total de requisições")
plt.ylabel("Tempo em segundos")
plt.plot(x_times_LFU_GBC)

In [ ]:
df_times_LFU_GBC = x_times_LFU_GBC.describe().transpose()
df_times_LFU_GBC['algo'] = 'LFU_GBC'

x_times_LFU_GBC.describe()

In [ ]:
ax = pd.concat([df_times_LRU, df_times_LFU, df_times_LRU_DT, df_times_LFU_DT,
           df_times_LRU_RF, df_times_LFU_RF, df_times_LRU_GBC, df_times_LFU_GBC]).plot.bar(x="algo", y="mean", log=True, figsize=(20, 16))

ax.set_title("Tempo médio de execução dos algoritmos", fontsize=18)
ax.set_xlabel("Sistemas propostos", fontsize=16)
ax.set_ylabel("Tempo médio (em log)", fontsize=16)

In [ ]:
pd.concat([df_times_LRU, df_times_LFU, df_times_LRU_DT, df_times_LFU_DT,
           df_times_LRU_GBC, df_times_LFU_GBC]).plot.bar(x="algo", y="mean", figsize=(10, 8))

In [ ]:
df_mean_time = pd.concat([df_times_LRU, df_times_LFU, df_times_LRU_DT, df_times_LFU_DT,
                         df_times_LRU_RF, df_times_LFU_RF, df_times_LRU_GBC, df_times_LFU_GBC])

In [ ]:
df_mean_time

### Conclusão

Neste trabalho foi proposto a utilização de três algoritmos de aprendizagem de máquina para classificação dos dados antes de um algoritmo de cache. A utilização prévia deles demostrou ser uma técnica eficaz para maximar o _hit ratio_ conseguindo classificar de maneira satisfatória a popularidade de um item no futuro e quais itens precisam passar pelo algoritmo de cache.
O sistema proposto utilizando _Decision Tree_ + LRU foi o melhor, conseguindo um _hit ratio_ de 96.93% isso devido ao fato do algoritmo de _Decision Tree_ precisar rodar no LRU apenas 44.66% das requisições. Ele se saiu bem também no tempo médio para classificação de cada requisição, levando em média 0.000891s.